In [1]:
import os
os.chdir('D:\Master_Code_92589')

In [2]:
import pickle
import numpy as np
import csv
import pandas as pd
from collections import Counter
from ast import Str

In [3]:
# table[y][x]
table = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
        [ 0, 1, 0, 0, 0, 0,11, 0, 0, 0, 0,21],\
        [ 0, 2, 0, 0, 0, 0,12, 0, 0, 0, 0,22],\
        [ 0, 3, 0, 0, 0, 0,13, 0, 0, 0, 0,23],\
        [ 0, 4, 0, 0, 0, 0,14, 0, 0, 0, 0,24],\
        [ 0, 5, 0, 0, 0, 0,15, 0, 0, 0, 0,25],\
        [ 0, 6, 0, 0, 0, 0,16, 0, 0, 0, 0,26],\
        [ 0, 7, 0, 0, 0, 0,17, 0, 0, 0, 0,27],\
        [ 0, 8, 0, 0, 0, 0,18, 0, 0, 0, 0,28],\
        [ 0, 9, 0, 0, 0, 0,19, 0, 0, 0, 0,29],\
        [ 0,10, 0, 0, 0, 0,20, 0, 0, 0, 0,30],\
        [ 0,31,32,33,34,35,36,37,38,39,40,41] ], dtype = int)

def class_to_coordinate(a):
    x = np.argwhere(table == a)[0][1]
    y = np.argwhere(table == a)[0][0]
    coordinate = [x,y]
    return coordinate

def uuid_to_b_num(s):
    if s[0:30] == 'e20a39f4-73f5-4bc4-a12f-17d1ad' :
        uuid_num = int(s[-1])
    else :
        uuid_num = 0
    return uuid_num

## 說明：
- 透過當初在每個位置收的 BLE_data.csv，產生wireless fingerprint training set
，一共生成三個檔案：
![](https://i.imgur.com/YBkSogr.png)

- max_rssi.pickle 跟 min_rssi.pickle 分別儲存每支手機的 RSSI 最大/最小值，以便進行 MIN-MAX Normalization

## 參數:
- 無


In [4]:
wireless_fingerprint = pd.DataFrame(columns =['label', 'Beacon_1', 'Beacon_2', 'Beacon_3', 'Beacon_4', 'Beacon_5', 'Beacon_6', 'Beacon_7'])

phone_rssi_list = {'hTCU11':[]}
min_rssi = { 'hTCU11':0,}
max_rssi = { 'hTCU11':0,}

for phone in ['hTCU11']:
    path = f'./Data/0318_92589_train/{phone}/'
    file_list = [f for f in os.listdir(path) if f[len(f)-3:] == 'csv']

    for rssi_file in file_list:
        df = pd.read_csv(f'{path+rssi_file}', delimiter=',')
        phone_rssi_list[phone] += df['RSSI'].to_list()

    min_rssi[phone] = min(phone_rssi_list[phone])
    max_rssi[phone] = max(phone_rssi_list[phone])
    print(min_rssi[phone],max_rssi[phone])

    for rssi_file in file_list:
        #position = table[int(rssi_file[1:2])][int(rssi_file[0])]
        position = rssi_file[0:len(rssi_file)-4]
        print(rssi_file, position)

        df = pd.read_csv(f'{path+rssi_file}', delimiter=',')
        df['UUID'] = df['UUID'].apply(uuid_to_b_num)
        #print(df[0:7])
        rssi = [np.empty((8),dtype=Str)]
        rssi[0][:] = np.nan
        rssi[0][0] = position
        row = 0

        for i in range(len(df) - 1):
            curr = df.iloc[i]['Time']
            next = df.iloc[i+1]['Time']

            rssi_idx = int(df.iloc[i]['UUID'])

            rssi_idx = int(df.iloc[i]['UUID'])
            if rssi_idx > 0 :
                rssi[row][rssi_idx] = (df.iloc[i]['RSSI']-min_rssi[phone])/(max_rssi[phone]-min_rssi[phone])

            if curr != next:  
                row += 1
                rssi.append(np.empty((8),dtype=Str))
                rssi[row][:] = np.nan
                rssi[row][0] = position

        rssi_idx = int(df.iloc[len(df) - 1]['UUID'])
        #print(rssi_idx)
        if rssi_idx > 0 :
            rssi[row][rssi_idx] = (df.iloc[len(df) - 1]['RSSI']-min_rssi[phone])/(max_rssi[phone]-min_rssi[phone])

        print(rssi)

        new_row = pd.DataFrame(np.array(rssi), columns =['label', 'Beacon_1', 'Beacon_2', 'Beacon_3', 'Beacon_4', 'Beacon_5', 'Beacon_6', 'Beacon_7'])
        wireless_fingerprint = wireless_fingerprint.append(new_row, ignore_index=True)
print(wireless_fingerprint.dtypes)
wireless_fingerprint = wireless_fingerprint.interpolate()
print(wireless_fingerprint)

-101 -46
1001.csv 1001
[array(['1001', 0.34545454545454546, 0.4, 0.7818181818181819,
       0.2909090909090909, 0.7636363636363637, nan, 0.41818181818181815],
      dtype=object), array(['1001', 0.34545454545454546, 0.43636363636363634,
       0.7818181818181819, 0.2909090909090909, 0.7636363636363637, nan,
       0.4], dtype=object), array(['1001', 0.34545454545454546, 0.45454545454545453,
       0.7454545454545455, 0.5454545454545454, 0.7636363636363637, nan,
       0.10909090909090909], dtype=object), array(['1001', 0.41818181818181815, 0.43636363636363634,
       0.7272727272727273, 0.3090909090909091, 0.5454545454545454, nan,
       0.43636363636363634], dtype=object), array(['1001', 0.4909090909090909, 0.4909090909090909, 0.6909090909090909,
       0.3090909090909091, 0.7090909090909091, nan, 0.3090909090909091],
      dtype=object), array(['1001', 0.4909090909090909, 0.43636363636363634,
       0.6909090909090909, 0.3090909090909091, 0.41818181818181815, nan,
       0.3272727272

TypeError: Cannot interpolate with all object-dtype columns in the DataFrame. Try setting at least one column to a numeric dtype.

In [5]:
# 儲存 min, max rssi 的檔案
import pickle
with open(f'./Data/0318_92589_train/min_rssi.pickle', 'wb') as f:
    pickle.dump(min_rssi, f)
with open(f'./Data/0318_92589_train/max_rssi.pickle', 'wb') as f:
    pickle.dump(max_rssi, f)

In [6]:
# 儲存 wireless fingerprint training set
wireless_fingerprint.set_index('label').to_csv(f'./Data/0318_92589_train/wireless_training_set.csv')